**<center> <span style="color:#0F52BA;font-family:serif; font-size:32px;"> KnowledgeKapture 1.1</span> </center>**

# Pre Processing And Stemming Function


In [28]:
# Import the necessary libraries and modules
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer


# nltk.download('stopwords')
# nltk.download('punkt')


# Define a function for text preprocessing
def Pre_processing_text(text):
    # Create a set of stopwords for English and add 'us' to it
    stop_words = set(stopwords.words('english') + ['us'])

    # Initialize a tokenizer to split text into words based on regular expressions
    tokenizer = RegexpTokenizer(r'\w+')

    # Tokenize the input text into a list of word tokens
    word_tokens = word_tokenize(text)

    # Remove stopwords from the tokenized text and convert to lowercase
    filtered_sentence_stopword = [w.lower() for w in word_tokens if not w.lower() in stop_words]

    # Tokenize the filtered text again to remove punctuation and create a list of word tokens
    filtered_sentence_punc = tokenizer.tokenize(" ".join(filtered_sentence_stopword))

    # Return the preprocessed text as a string
    return " ".join(filtered_sentence_punc)


In [29]:
# Import the Porter Stemmer from the NLTK library
from nltk.stem import PorterStemmer


# Define a function for stemming sentences
def Stemming_sentences(text):
    # Create an instance of the Porter Stemmer
    ps = PorterStemmer()

    # Split the input text into words and apply stemming to each word
    stemming_list = [ps.stem(w) for w in text.split()]

    # Join the stemmed words back into a single string
    return " ".join(stemming_list)



# Crawling

# Reading All Doc and Their Names

In [30]:
# crawling
import os
import docx
from pdfquery import PDFQuery

path_of_files = r'Docalpha'
entries = os.listdir(fr'{path_of_files}')
doc_dict_alpha = {}

for name in entries:

    if name[-3:] == 'txt':
        with open(fr'{path_of_files}\{name}', "r", encoding='utf8') as w:
            doc_dict_alpha[name] = w.readlines()

    elif name[-4:] == 'docx':

        doc = docx.Document(fr'{path_of_files}\{name}')

        new_list = []
        # Read the content
        for paragraph in doc.paragraphs:
            text = paragraph.text
            new_list.append(text)
        doc_dict_alpha[name] = new_list


    elif name[-3:] == 'pdf':

        pdf = PDFQuery(fr'{path_of_files}\{name}')
        pdf.load()

        # Use CSS-like selectors to locate the elements
        text_elements = pdf.pq('LTTextLineHorizontal')

        # Extract the text from the elements
        text = [t.text for t in text_elements]
        doc_dict_alpha[name] = text

# Stemming and Preprocessing The Doc

In [31]:
# Create an empty dictionary to store preprocessed and stemmed text
doc_dict_beta = {}

# Iterate through the entries in the original 'doc_dict_alpha' dictionary
for i in doc_dict_alpha:
    # Initialize an empty list to store preprocessed and stemmed lines
    res = []
    # Iterate through each line in the current document
    for sub in doc_dict_alpha[i]:
        # Remove newline characters from each line and append to 'res'
        res.append(sub.replace("\n", ""))

    # Join the preprocessed lines into a single string, strip any leading/trailing whitespace,
    # preprocess the text, and then stem it
    preprocessed_and_stemmed_text = Stemming_sentences(Pre_processing_text(" ".join(res).strip()))

    # Store the preprocessed and stemmed text in the 'doc_dict_beta' dictionary
    doc_dict_beta[i] = preprocessed_and_stemmed_text

# The 'doc_dict_beta' dictionary now contains preprocessed and stemmed text for each document.
doc_dict_beta

{'file_1.txt': 'elast search big data text mine tool',
 'file_2.txt': 'elast search help understand data gog',
 'file_3.txt': 'big data use big inform',
 'file_4.txt': 'anyth els much',
 'file_5.txt': 'onlin freind father',
 'test.docx': 'elast search big data text mine tool ttttttt ytjyt jytjti trhtrhr',
 'test3.pdf': 'elast search big data text mine tool ttttttt'}

# Inverted index Terms

In [32]:
inverted_index_list_word = list()

for i in doc_dict_beta:
    
    inverted_index_list_word.extend(doc_dict_beta[i].split())
    
inverted_index_list_word = list(set(inverted_index_list_word))
inverted_index_list_word

['freind',
 'onlin',
 'father',
 'use',
 'jytjti',
 'inform',
 'search',
 'tool',
 'anyth',
 'text',
 'els',
 'mine',
 'gog',
 'big',
 'trhtrhr',
 'data',
 'ytjyt',
 'ttttttt',
 'understand',
 'elast',
 'much',
 'help']

# Inverted Index

In [33]:
inverted_index_dict_wrod = {
    
    
}


for i in inverted_index_list_word:
    
    doc_word_list = []
    
    for c in doc_dict_beta:
        
        if i in doc_dict_beta[c]:
            doc_word_list.append(c)
            
    inverted_index_dict_wrod[i] = doc_word_list
inverted_index_dict_wrod

{'freind': ['file_5.txt'],
 'onlin': ['file_5.txt'],
 'father': ['file_5.txt'],
 'use': ['file_3.txt'],
 'jytjti': ['test.docx'],
 'inform': ['file_3.txt'],
 'search': ['file_1.txt', 'file_2.txt', 'test.docx', 'test3.pdf'],
 'tool': ['file_1.txt', 'test.docx', 'test3.pdf'],
 'anyth': ['file_4.txt'],
 'text': ['file_1.txt', 'test.docx', 'test3.pdf'],
 'els': ['file_4.txt'],
 'mine': ['file_1.txt', 'test.docx', 'test3.pdf'],
 'gog': ['file_2.txt'],
 'big': ['file_1.txt', 'file_3.txt', 'test.docx', 'test3.pdf'],
 'trhtrhr': ['test.docx'],
 'data': ['file_1.txt', 'file_2.txt', 'file_3.txt', 'test.docx', 'test3.pdf'],
 'ytjyt': ['test.docx'],
 'ttttttt': ['test.docx', 'test3.pdf'],
 'understand': ['file_2.txt'],
 'elast': ['file_1.txt', 'file_2.txt', 'test.docx', 'test3.pdf'],
 'much': ['file_4.txt'],
 'help': ['file_2.txt']}

In [34]:
import json
with open('result.json', 'w') as fp:
    json.dump(inverted_index_dict_wrod, fp)
new_inverted_index = json.load(open("result.json")) 
new_inverted_index

{'freind': ['file_5.txt'],
 'onlin': ['file_5.txt'],
 'father': ['file_5.txt'],
 'use': ['file_3.txt'],
 'jytjti': ['test.docx'],
 'inform': ['file_3.txt'],
 'search': ['file_1.txt', 'file_2.txt', 'test.docx', 'test3.pdf'],
 'tool': ['file_1.txt', 'test.docx', 'test3.pdf'],
 'anyth': ['file_4.txt'],
 'text': ['file_1.txt', 'test.docx', 'test3.pdf'],
 'els': ['file_4.txt'],
 'mine': ['file_1.txt', 'test.docx', 'test3.pdf'],
 'gog': ['file_2.txt'],
 'big': ['file_1.txt', 'file_3.txt', 'test.docx', 'test3.pdf'],
 'trhtrhr': ['test.docx'],
 'data': ['file_1.txt', 'file_2.txt', 'file_3.txt', 'test.docx', 'test3.pdf'],
 'ytjyt': ['test.docx'],
 'ttttttt': ['test.docx', 'test3.pdf'],
 'understand': ['file_2.txt'],
 'elast': ['file_1.txt', 'file_2.txt', 'test.docx', 'test3.pdf'],
 'much': ['file_4.txt'],
 'help': ['file_2.txt']}

# Query

In [35]:
query = "Elastic search in Big data "

stem_query = Stemming_sentences(Pre_processing_text(query))
stem_query

'elast search big data'

# The Documents That Contain The Words In Query

In [36]:
set_of_doc_query = list()

for i in new_inverted_index:
    if i in stem_query.split():
        set_of_doc_query.extend(new_inverted_index[i])

set_of_doc_query = list(set(set_of_doc_query))
set_of_doc_query

['test.docx', 'file_3.txt', 'file_2.txt', 'test3.pdf', 'file_1.txt']

In [37]:
doc_dict = {
    
    
    
}


for i in sorted(set_of_doc_query):
    doc_dict[i] = doc_dict_beta[i]
    
doc_dict

{'file_1.txt': 'elast search big data text mine tool',
 'file_2.txt': 'elast search help understand data gog',
 'file_3.txt': 'big data use big inform',
 'test.docx': 'elast search big data text mine tool ttttttt ytjyt jytjti trhtrhr',
 'test3.pdf': 'elast search big data text mine tool ttttttt'}

# Not Necessary Step for Doc

In [38]:
filtered_stopwords_punc_doc_dict = doc_dict.copy()

for  i in filtered_stopwords_punc_doc_dict:
    print(f" The {i} : {filtered_stopwords_punc_doc_dict[i]} ")
    
print("*" *30)
print(f"Query -- >  {query}")

 The file_1.txt : elast search big data text mine tool 
 The file_2.txt : elast search help understand data gog 
 The file_3.txt : big data use big inform 
 The test.docx : elast search big data text mine tool ttttttt ytjyt jytjti trhtrhr 
 The test3.pdf : elast search big data text mine tool ttttttt 
******************************
Query -- >  Elastic search in Big data 


# Not Necessary Step for Doc

In [39]:
stemming_dict = dict()

for i in filtered_stopwords_punc_doc_dict:
    
    stemming_dict[i] = Stemming_sentences(filtered_stopwords_punc_doc_dict[i])

stem_query = Stemming_sentences(query)    
    

for  i in stemming_dict:
    print(f" The {i} : {stemming_dict[i]} ")
    
print("*" *30)
print(f"Query -- >  {stem_query}")

 The file_1.txt : elast search big data text mine tool 
 The file_2.txt : elast search help understand data gog 
 The file_3.txt : big data use big inform 
 The test.docx : elast search big data text mine tool ttttttt ytjyt jytjti trhtrhr 
 The test3.pdf : elast search big data text mine tool ttttttt 
******************************
Query -- >  elast search in big data


# All Terms in docs


In [40]:
set_of_words = set()
list_of_words = []
for i in stemming_dict.values():
    
    for c in set(i.split(" ")):
        set_of_words.add(c)
        list_of_words.append(c)
set_of_words = list(set_of_words)
set_of_words

['data',
 'jytjti',
 'ytjyt',
 'ttttttt',
 'mine',
 'inform',
 'gog',
 'search',
 'trhtrhr',
 'understand',
 'tool',
 'elast',
 'text',
 'big',
 'help',
 'use']

In [41]:
list_of_words

['data',
 'mine',
 'search',
 'elast',
 'tool',
 'text',
 'big',
 'data',
 'gog',
 'search',
 'understand',
 'elast',
 'help',
 'inform',
 'data',
 'big',
 'use',
 'data',
 'jytjti',
 'ytjyt',
 'ttttttt',
 'mine',
 'search',
 'trhtrhr',
 'elast',
 'tool',
 'text',
 'big',
 'data',
 'ttttttt',
 'mine',
 'search',
 'elast',
 'tool',
 'text',
 'big']

# df IDF Table

In [42]:
import pandas as pd
import numpy as np
docfreq_idf_dict = {
    
    "terms" : set_of_words,
    
    "DocFreq" : [i * 0 for i in range(len(set_of_words))],
    
    "IDF" : [i * 0 for i in range(len(set_of_words))]
    
}

docfreq_idf_df = pd.DataFrame(docfreq_idf_dict).sort_values(by = 'terms').reset_index(drop=True)
docfreq_idf_df

,terms,DocFreq,IDF
0,big,0,0
1,data,0,0
2,elast,0,0
3,gog,0,0
4,help,0,0
5,inform,0,0
6,jytjti,0,0
7,mine,0,0
8,search,0,0
9,text,0,0


# Calculate DocFreq-IDF

In [43]:
for i in list(docfreq_idf_df.index):
    
    doc_freq_term = list_of_words.count(docfreq_idf_df.loc[i,"terms"])
    
    docfreq_idf_df.loc[i,"DocFreq"] = doc_freq_term
    docfreq_idf_df.loc[i,"IDF"]  = np.log10(len(doc_dict) / doc_freq_term)

docfreq_idf_df.index =  docfreq_idf_df.index +1 
docfreq_idf_df

C:\Users\HOl\AppData\Local\Temp\ipykernel_10500\3414053815.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.09691001300805642' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  docfreq_idf_df.loc[i,"IDF"]  = np.log10(len(doc_dict) / doc_freq_term)


,terms,DocFreq,IDF
1,big,4,0.096910
2,data,5,0.000000
3,elast,4,0.096910
4,gog,1,0.698970
5,help,1,0.698970
6,inform,1,0.698970
7,jytjti,1,0.698970
8,mine,3,0.221849
9,search,4,0.096910
10,text,3,0.221849


# Tf Table

In [44]:
array_for_weight_freq_df = np.zeros((len(doc_dict)+1,len(docfreq_idf_df.index)))
array_for_weight_freq_df

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [45]:
df_for_weight_freq_df = pd.DataFrame(array_for_weight_freq_df,
                                     columns = [f"T{i}" for i in docfreq_idf_df.index],index  = [i for i in doc_dict] + ['Q'])
df_for_weight_freq_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16
file_1.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
file_2.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
file_3.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test.docx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test3.pdf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Calculate Tf Table

In [46]:
df_for_freq_df = df_for_weight_freq_df.copy()

for i in df_for_weight_freq_df.index:
    
    
    if i == 'Q':
        continue
    
    for c in df_for_weight_freq_df.columns:
        
        term_freq_doc = stemming_dict[i].split(" ").count(docfreq_idf_df.loc[int(c[1:]),"terms"])
        
        df_for_freq_df.loc[i,c] = term_freq_doc
        

for c in df_for_weight_freq_df.columns:
    
    term_freq_query = stem_query.split(" ").count(docfreq_idf_df.loc[int(c[1:]),"terms"])
    
    df_for_freq_df.loc["Q",c] = term_freq_query


df_for_freq_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16
file_1.txt,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
file_2.txt,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
file_3.txt,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
test.docx,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
test3.pdf,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
Q,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Calculate Weight Table

In [47]:
df_for_weight_df = df_for_weight_freq_df.copy()

for i in df_for_weight_freq_df.index:
    
    
    if i == 'Q':
        continue
    
    for c in df_for_weight_freq_df.columns:
        
        term_freq_doc = stemming_dict[i].split(" ").count(docfreq_idf_df.loc[int(c[1:]),"terms"])
        
        idf = docfreq_idf_df.loc[int(c[1:]),"IDF"]
        
        df_for_weight_df.loc[i,c] = term_freq_doc * idf
        

for c in df_for_weight_freq_df.columns:
    
    term_freq_query = stem_query.split(" ").count(docfreq_idf_df.loc[int(c[1:]),"terms"])
    
    idf = docfreq_idf_df.loc[int(c[1:]),"IDF"]
    
    df_for_weight_df.loc["Q",c] = term_freq_query * idf


df_for_weight_df

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16
file_1.txt,0.09691,0.0,0.09691,0.00000,0.00000,0.00000,0.00000,0.221849,0.09691,0.221849,0.221849,0.00000,0.00000,0.00000,0.00000,0.00000
file_2.txt,0.00000,0.0,0.09691,0.69897,0.69897,0.00000,0.00000,0.000000,0.09691,0.000000,0.000000,0.00000,0.00000,0.69897,0.00000,0.00000
file_3.txt,0.19382,0.0,0.00000,0.00000,0.00000,0.69897,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.69897,0.00000
test.docx,0.09691,0.0,0.09691,0.00000,0.00000,0.00000,0.69897,0.221849,0.09691,0.221849,0.221849,0.69897,0.39794,0.00000,0.00000,0.69897
test3.pdf,0.09691,0.0,0.09691,0.00000,0.00000,0.00000,0.00000,0.221849,0.09691,0.221849,0.221849,0.00000,0.39794,0.00000,0.00000,0.00000
Q,0.09691,0.0,0.09691,0.00000,0.00000,0.00000,0.00000,0.000000,0.09691,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000


In [48]:

methods_result = ['inner',"cosine","jaccard","dice"]


df_result = pd.DataFrame(np.zeros((len(doc_dict),4)),index = doc_dict.keys() ,columns = methods_result)
df_result

,inner,cosine,jaccard,dice
file_1.txt,0.0,0.0,0.0,0.0
file_2.txt,0.0,0.0,0.0,0.0
file_3.txt,0.0,0.0,0.0,0.0
test.docx,0.0,0.0,0.0,0.0
test3.pdf,0.0,0.0,0.0,0.0


# Result Table

In [49]:

methods_result = ['inner',"cosine","jaccard","dice"]


df_result = pd.DataFrame(np.zeros((len(doc_dict),4)),index = doc_dict.keys() ,columns = methods_result)
df_result

,inner,cosine,jaccard,dice
file_1.txt,0.0,0.0,0.0,0.0
file_2.txt,0.0,0.0,0.0,0.0
file_3.txt,0.0,0.0,0.0,0.0
test.docx,0.0,0.0,0.0,0.0
test3.pdf,0.0,0.0,0.0,0.0


# Inner Product

In [50]:
# FIRST METHOD , inner product ∑(𝒅𝒊𝒌.𝒒𝒌)

# Calculate the inner product between query and documents weights
query_weights = df_for_weight_df.loc['Q']
doc_weights = df_for_weight_df.drop('Q')

inner_product = doc_weights.dot(query_weights)

df_result["inner"]=inner_product

df_result

,inner,cosine,jaccard,dice
file_1.txt,0.028175,0.0,0.0,0.0
file_2.txt,0.018783,0.0,0.0,0.0
file_3.txt,0.018783,0.0,0.0,0.0
test.docx,0.028175,0.0,0.0,0.0
test3.pdf,0.028175,0.0,0.0,0.0


# Cosine


In [51]:

# SECONDE METHOD , cosine

query_tf = df_for_freq_df.loc['Q']
doc_tf = df_for_freq_df.drop('Q')

#Numerator :
dot_products = doc_tf.dot(query_tf)

#denominator :
#length for doc and query
len_query = np.sqrt(np.sum(np.square(query_tf)))

len_docs = np.sqrt(np.sum(np.square(doc_tf), axis=1))

cosine = dot_products / (len_docs * len_query)

df_result["cosine"]=cosine

df_result

,inner,cosine,jaccard,dice
file_1.txt,0.028175,0.755929,0.0,0.0
file_2.txt,0.018783,0.612372,0.0,0.0
file_3.txt,0.018783,0.566947,0.0,0.0
test.docx,0.028175,0.603023,0.0,0.0
test3.pdf,0.028175,0.707107,0.0,0.0


# Jaccard

In [52]:

# THIRD METHOD , jaccard : i𝒏𝒏𝒆𝒓 𝒑𝒓𝒐𝒅𝒖𝒄𝒕/(∑𝒅𝟐ⅈ𝒌. + ∑𝒒𝟐𝒌−𝒊𝒏𝒏𝒆𝒓 𝒑𝒓𝒐𝒅𝒖𝒄𝒕)

squared_weights_query= np.sum(np.square(query_weights))
sum_squared_weights = np.square(doc_weights).sum(axis=1)
Jaccard = inner_product / (squared_weights_query + sum_squared_weights - inner_product)
df_result["jaccard"]=Jaccard
df_result

,inner,cosine,jaccard,dice
file_1.txt,0.028175,0.755929,0.160242,0.0
file_2.txt,0.018783,0.612372,0.012574,0.0
file_3.txt,0.018783,0.566947,0.018342,0.0
test.docx,0.028175,0.603023,0.015654,0.0
test3.pdf,0.028175,0.707107,0.084309,0.0


# dice

In [53]:


#FORTH METHOD , dice
#Numerator :
dot_products = doc_tf.dot(query_tf)

#denominator :
sum_squared_query= np.sum(np.square(query_tf))
sum_squared_doc = np.square(doc_tf).sum(axis=1)

#final equation
dice = 2*dot_products / (sum_squared_doc + sum_squared_query)
df_result["dice"]=dice
df_result

,inner,cosine,jaccard,dice
file_1.txt,0.028175,0.755929,0.160242,0.727273
file_2.txt,0.018783,0.612372,0.012574,0.600000
file_3.txt,0.018783,0.566947,0.018342,0.545455
test.docx,0.028175,0.603023,0.015654,0.533333
test3.pdf,0.028175,0.707107,0.084309,0.666667


# Sorting Table

In [54]:
x = list(df_result.columns)

sorted_df2 = df_result.sort_values(by=x, ascending=False)

sorted_df2

,inner,cosine,jaccard,dice
file_1.txt,0.028175,0.755929,0.160242,0.727273
test.docx,0.028175,0.603023,0.015654,0.533333
test3.pdf,0.028175,0.707107,0.084309,0.666667
file_2.txt,0.018783,0.612372,0.012574,0.600000
file_3.txt,0.018783,0.566947,0.018342,0.545455
